In [2]:
import streamlit as st
import os
import pandas as pd
from PIL import Image

In [3]:
dir = './dataSources/'
labels = os.listdir(dir + '/Garbage classification/Garbage classification/')

In [4]:
dicts = []

for label in labels:
    directory = os.path.join(dir + '/Garbage classification/Garbage classification/', label)
    samples = os.listdir(directory)

    x = {'Type': label, 'Samples': len(samples)}
    dicts.append(x)


df = pd.DataFrame.from_dict(dicts)
df

,Type,Samples
0,paper,594
1,metal,410
2,cardboard,403
3,trash,137
4,glass,501
5,plastic,482
